In [7]:
import pypelines.ts_forecast_pipeline as pipe
from pypelines import utils
import pandas as pd
airline = pd.read_csv("pypelines/datasets/timeseries_forecast/airline.csv")
longley = pd.read_csv("pypelines/datasets/timeseries_forecast/longley.csv")

In [14]:
from sktime.datasets import load_longley
_, y = load_longley()

In [18]:
from sktime.forecasting.var import VAR
from sktime.datasets import load_longley
_, y = load_longley()
forecaster = VAR()  
forecaster.fit(y)  

y_pred = forecaster.predict(fh=[1,2,3])  

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "__init__.pxd", line 942, in numpy.import_array
RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Kreshnaa Raam\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Kreshnaa Raam\AppData\Local\Temp\ipykernel_680220\2318451469.py", line 5, in <module>
    forecaster.fit(y)
  File "c:\Users\Kreshnaa Raam\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\forecasting\base\_base.py", line 348, in fit
    self._fit(y=y_inner, X=X_inner, fh=f

In [4]:
tsf = pipe.TSForecastPipeline(data = "airline",models=['SARIMAX'],
                 date_column = 'Date',
                 date_format = "%Y-%m",
                 frequency = "M",
                 forecast_horizon = [1,2,3])

In [9]:
tsf = pipe.TSForecastPipeline(data = "longley",models=['VAR'],
                 date_column = 'YEAR',
                 date_format = "%Y",
                 frequency = "Y",
                 forecast_horizon = [1,2,3])

In [11]:
tsf.code_to_clipboard()

In [13]:
x_train_preprocessed

,Obs,TOTEMP,GNPDEFL,GNP,UNEMP,ARMED,POP
YEAR,,,,,,,
1947,1,60323,83.0,234289,2356,1590,107608
1948,2,61122,88.5,259426,2325,1456,108632
1949,3,60171,88.2,258054,3682,1616,109773
1950,4,61187,89.5,284599,3351,1650,110929
1951,5,63221,96.2,328975,2099,3099,112075
1952,6,63639,98.1,346999,1932,3594,113270
1953,7,64989,99.0,365385,1870,3547,115094
1954,8,63761,100.0,363112,3578,3350,116219
1955,9,66019,101.2,397469,2904,3048,117388


In [12]:


from sktime import *
from sklearn.metrics import mean_squared_error


import pandas as pd
import numpy as np


date_col = 'YEAR'
features = list(longley.columns)
feature_df = longley[features]

x_train = longley[features]
x_train_preprocessed = x_train
x_train_preprocessed[date_col] = pd.to_datetime(x_train_preprocessed[date_col],format = "%Y")
x_train_preprocessed = x_train_preprocessed.set_index(date_col)
x_train_preprocessed.index = x_train_preprocessed.index.to_period(freq = "Y")

model_comparison_list = []

##### End of Data Processing Pipeline #####


##### Model Pipeline for VAR #####

from sktime.forecasting.var  import VAR
from sktime.forecasting.model_selection import ExpandingWindowSplitter,ForecastingGridSearchCV,ExpandingWindowSplitter
from sklearn.metrics import mean_squared_error,make_scorer,r2_score,explained_variance_score
import matplotlib.pyplot as plt
cv = ExpandingWindowSplitter(fh=[1, 2, 3])

VAR_param_grid = {
"trend": ['c'],
"ic": ['aic'],
}


VAR_model = VAR()

VAR_gscv = ForecastingGridSearchCV(
    forecaster=VAR_model,
    param_grid=VAR_param_grid,
    cv=cv,
    verbose=1)
VAR_gscv.fit(x_train_preprocessed)

# get the prediction on the test data
VAR_y_pred = VAR_gscv.predict(fh=[1, 2, 3])
##### End of Model Pipeline for VAR #####
##### Model Comparison #####




Fitting 4 folds for each of 1 candidates, totalling 4 fits


c:\Users\Kreshnaa Raam\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\forecasting\model_evaluation\_functions.py:413: FitFailedWarning: 
                In evaluate, fitting of forecaster VAR failed,
                you can set error_score='raise' in evaluate to see
                the exception message. Fit failed for len(y_train)=10.
                The score will be set to nan.
                Failed forecaster with parameters: VAR(ic='aic').
                
  result = _evaluate_window(
c:\Users\Kreshnaa Raam\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\forecasting\model_evaluation\_functions.py:413: FitFailedWarning: 
                In evaluate, fitting of forecaster VAR failed,
                you can set error_score='raise' in evaluate to see
                the exception message. Fit failed for len(y_train)=11.
                The score will be set to nan.
                Failed forecaster with parameters: VAR(ic='aic').
                
 

NotFittedError: All fits of forecaster failed,
                set error_score='raise' to see the exceptions.
                Failed forecaster: VAR()

In [6]:
SARIMAX_y_pred

,Passengers
1961-01,432.929169
1961-02,434.257751
1961-03,435.586334
